In [ ]:
!pip install pennylane

In [ ]:
import numpy as np
import pennylane as qml
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
from collections import Counter

In [ ]:
df = pd.read_csv("DatasetExos.csv" ,delimiter="," )
df

,ep (ms),Acc_x,Acc_y,Acc_z,Gyro_x,Gyro_y,Gyro_z,ID,Exercise,Category,Set
0,2019-01-11 15:08:05.200,0.013500,0.977000,-0.071000,-1.8904,2.4392,0.9388,B,bench,heavy,30.0
1,2019-01-11 15:08:05.400,-0.001500,0.970500,-0.079500,-1.6826,-0.8904,2.1708,B,bench,heavy,30.0
2,2019-01-11 15:08:05.600,0.001333,0.971667,-0.064333,2.5608,-0.2560,-1.4146,B,bench,heavy,30.0
3,2019-01-11 15:08:05.800,-0.024000,0.957000,-0.073500,8.0610,-4.5244,-2.0730,B,bench,heavy,30.0
4,2019-01-11 15:08:06.000,-0.028000,0.957667,-0.115000,2.4390,-1.5486,-3.6098,B,bench,heavy,30.0
...,...,...,...,...,...,...,...,...,...,...,...
9004,2019-01-20 17:33:27.000,-0.048000,-1.041500,-0.076500,1.4146,-5.6218,0.2926,E,row,medium,40.0
9005,2019-01-20 17:33:27.200,-0.037000,-1.030333,-0.053333,-2.7684,-0.5854,2.2440,E,row,medium,40.0
9006,2019-01-20 17:33:27.400,-0.060000,-1.031000,-0.082000,2.8416,-5.1342,-0.1220,E,row,medium,40.0
9007,2019-01-20 17:33:27.600,-0.038667,-1.025667,-0.044667,-0.2318,0.2562,1.1220,E,row,medium,40.0


In [ ]:
df.columns

Index(['ep (ms)', 'Acc_x', 'Acc_y', 'Acc_z', 'Gyro_x', 'Gyro_y', 'Gyro_z',
       'ID', 'Exercise', 'Category', 'Set'],
      dtype='object')

In [ ]:
df = df.drop(columns=["ep (ms)" , "Exercise"  , "ID" , "Category"])
df

,Acc_x,Acc_y,Acc_z,Gyro_x,Gyro_y,Gyro_z,Set
0,0.013500,0.977000,-0.071000,-1.8904,2.4392,0.9388,30.0
1,-0.001500,0.970500,-0.079500,-1.6826,-0.8904,2.1708,30.0
2,0.001333,0.971667,-0.064333,2.5608,-0.2560,-1.4146,30.0
3,-0.024000,0.957000,-0.073500,8.0610,-4.5244,-2.0730,30.0
4,-0.028000,0.957667,-0.115000,2.4390,-1.5486,-3.6098,30.0
...,...,...,...,...,...,...,...
9004,-0.048000,-1.041500,-0.076500,1.4146,-5.6218,0.2926,40.0
9005,-0.037000,-1.030333,-0.053333,-2.7684,-0.5854,2.2440,40.0
9006,-0.060000,-1.031000,-0.082000,2.8416,-5.1342,-0.1220,40.0
9007,-0.038667,-1.025667,-0.044667,-0.2318,0.2562,1.1220,40.0


In [ ]:
for i in df.columns:
    if i not in ["Category"]:
        df[i] = pd.to_numeric(df[i])
df

,Acc_x,Acc_y,Acc_z,Gyro_x,Gyro_y,Gyro_z,Set
0,0.013500,0.977000,-0.071000,-1.8904,2.4392,0.9388,30.0
1,-0.001500,0.970500,-0.079500,-1.6826,-0.8904,2.1708,30.0
2,0.001333,0.971667,-0.064333,2.5608,-0.2560,-1.4146,30.0
3,-0.024000,0.957000,-0.073500,8.0610,-4.5244,-2.0730,30.0
4,-0.028000,0.957667,-0.115000,2.4390,-1.5486,-3.6098,30.0
...,...,...,...,...,...,...,...
9004,-0.048000,-1.041500,-0.076500,1.4146,-5.6218,0.2926,40.0
9005,-0.037000,-1.030333,-0.053333,-2.7684,-0.5854,2.2440,40.0
9006,-0.060000,-1.031000,-0.082000,2.8416,-5.1342,-0.1220,40.0
9007,-0.038667,-1.025667,-0.044667,-0.2318,0.2562,1.1220,40.0


In [ ]:
df = df.reset_index()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9009 entries, 0 to 9008
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   index   9009 non-null   int64  
 1   Acc_x   9005 non-null   float64
 2   Acc_y   9009 non-null   float64
 3   Acc_z   9008 non-null   float64
 4   Gyro_x  9008 non-null   float64
 5   Gyro_y  9002 non-null   float64
 6   Gyro_z  9009 non-null   float64
 7   Set     8999 non-null   float64
dtypes: float64(7), int64(1)
memory usage: 563.2 KB


In [ ]:
df = df.dropna()
df = df.drop(columns=["index"])

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8990 entries, 0 to 9008
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Acc_x   8990 non-null   float64
 1   Acc_y   8990 non-null   float64
 2   Acc_z   8990 non-null   float64
 3   Gyro_x  8990 non-null   float64
 4   Gyro_y  8990 non-null   float64
 5   Gyro_z  8990 non-null   float64
 6   Set     8990 non-null   float64
dtypes: float64(7)
memory usage: 561.9 KB


In [ ]:
from sklearn.preprocessing import StandardScaler

Y = df["Set"]
X = df.drop(columns="Set")
scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
X_train

array([[-0.00939293, -1.29482109,  0.07765545, -1.66261847,  0.49020771,
         0.08600606],
       [-0.00935833, -0.87137017, -0.00396632, -2.25493298, -0.2317713 ,
         0.16835014],
       [-0.00958133,  0.92034967, -0.73004952, -0.53282804,  0.11358841,
         0.11497842],
       ...,
       [-0.00926744,  0.65108019,  1.55736294,  0.09128153,  0.01143351,
        -0.0084841 ],
       [-0.00960933,  0.62696651, -0.59284483,  0.66219847,  0.53954155,
         0.06503822],
       [-0.00952786, -1.20640424, -0.47466853, -0.15990393,  0.07396407,
         0.00327972]])

In [ ]:
y_train

,Set
7381,3.0
3854,35.0
2094,7.0
439,87.0
4038,12.0
...,...
5749,1.0
5205,68.0
5404,73.0
864,84.0


1. transform data into quantum states

In [ ]:
_qubits = 6
dev = qml.device('default.qubit', wires=_qubits)

def encode_data(data):
    encoded_states = []
    for point in data:
        encoded_state = np.zeros((_qubits,), dtype=int)
        for i in range(_qubits):
            if point[i] > 0.5:
                encoded_state[i] = 1
        encoded_states.append(encoded_state)
    return np.array(encoded_states)

# circuit to calculate distances
@qml.qnode(dev)
def quantum_distance(x, y):
    x = np.array(x, dtype=int)
    y = np.array(y, dtype=float)

    qml.BasisState(x, wires=range(_qubits))
    for i in range(_qubits):
        qml.RY(y[i] * np.pi, wires=i)

    return qml.expval(qml.Hermitian(np.diag([1, -1]), wires=0))

# calculate distances
def calculate_distances(X_train, X_test):
    X_train_enc = encode_data(X_train)
    X_test_enc = encode_data(X_test)

    distances = np.zeros((len(X_test_enc), len(X_train_enc)))
    for i, x_test in enumerate(X_test_enc):
        for j, x_train in enumerate(X_train_enc):
            distances[i, j] = quantum_distance(x_test, x_train)
    return distances

In [ ]:
encoded_X_train = encode_data(X_train)
encoded_X_test = encode_data(X_test)

In [ ]:
encoded_X_train

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       ...,
       [0, 1, 1, 0, 0, 0],
       [0, 1, 0, 1, 1, 0],
       [0, 0, 0, 0, 0, 0]])

In [ ]:
distances = calculate_distances(encoded_X_train, encoded_X_test)

print("Calculated Distances:")
for i, row in enumerate(distances):
    print(f"Distances from test point {i}: {row}")

In [ ]:
def get_best_neighbors(distances, k=6):
    best_neighbors = []
    for distance_row in distances:
        # Get the indices of the k smallest distances
        neighbors_indices = np.argsort(distance_row)[:k]
        best_neighbors.append(neighbors_indices)
    return best_neighbors

best_neighbors = get_best_neighbors(distances)
best_neighbors

In [ ]:
for i, neighbors in enumerate(best_neighbors):
    print(f"Best neighbors for test point {i}: {neighbors} with distances: {distances[i][neighbors]}")

In [ ]:
y_train

In [ ]:
def extract_classes(best_neighbors, y_train):
    predicted_classes = []
    for neighbors in best_neighbors:
      try :
        neighbor_classes = y_train.iloc[neighbors]
        most_common_class = Counter(neighbor_classes).most_common(1)[0][0]
        predicted_classes.append(most_common_class)
      except Exception as e:
        print(e)
    return predicted_classes
predicted_classes = extract_classes(best_neighbors, y_train)

In [ ]:
for i, (neighbors, predicted_class) in enumerate(zip(best_neighbors, predicted_classes)):
    print(f"Best neighbors for test point {i}: {neighbors} with predicted class: {predicted_class}")